In [1]:
# import liabries
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [2]:
#  src url and initial soup
# get url list for data scraping
url = 'https://www.who.int/csr/sars/country/en/'
bs0 = bs(requests.get(url).content,'html')
bs1 = bs0.find_all('a')
lst = list(map(lambda x: x if '/csr/sars/country/' in str(x) else 0, bs1))
who_main = 'http://www.who.int'
url_lst = list(map(lambda x: who_main+x,[lst[i]['href'] for i in range(len(lst)) if lst[i]!=0] ))

# url list to use for scraping
url_use = [i for i in url_lst if len(bs(requests.get(i).content,'html').find_all('tr'))!=0]

In [3]:
# normalize and retrieve list of date for df date column 
data_date = list(map(lambda x:re.findall('\d+_\d+_\d+',x)[0],url_use[3:]))
date_lst = ['2003_03_17','2003_03_18','2003_03_19']

#df date col data
data_date = date_lst+data_date


In [4]:
# define function to scrap data using url

def get_data(url,date):
    try:
        soup_table = bs(requests.get(url).content,'html').find_all('tr')
        rows = []
        for i in range(len(soup_table)):
            if '/b' in str(soup_table[i]):
                rows.append([j.text.replace(u'\xa0',u' ').strip('\n') for j in soup_table[i].find_all('b')])
            elif '/p' in str(soup_table[i]):
                rows.append([j.text.replace(u'\xa0',u' ').strip('\n') for j in soup_table[i].find_all('p')])
            elif '/td' in str(soup_table[i]):
                rows.append([j.text.replace(u'\xa0',u' ').strip('\n') for j in soup_table[i].find_all('td')]) 
        
        df = pd.DataFrame(data = rows[1:], columns=rows[0])
        df['Date'] = date
        result = [rows, df['Date']]
        return result
                # df.to_csv('/c/Users/sfy/Documents/Ironhack/Personal_Projects/data/%s')
    except:
        print('ValueError! Check source.',url)
        # raise ValueError('Check Column List Length')
        


In [132]:
# define function to clean data
def clean_col(x):
    cln_lst = x
    if type(cln_lst) == list:
        if len(cln_lst) != 0:
            for i in range(len(cln_lst[0])):
                if 'transmission' in cln_lst[0][i]:
                    cln_lst[0][i] = 'Local_Chain(s)_of_Transmission'
                elif 'Country' in cln_lst[0][i]:
                    cln_lst[0][i] = 'Country'
                elif ('Cumulative number' in cln_lst[0][i]) & ('Date for which cumulative number of cases is current' not in cln_lst[0][i]) == True:
                    cln_lst[0][i] = 'Cumulative_Number_Of_Case(s)'
                elif ('Number of deaths' in cln_lst[0][i]) & ('Final Status/ Number of deaths' not in cln_lst[0][i]) == True:
                    cln_lst[0][i] = 'Number_Of_Deaths'
                elif ('Number recovered' in cln_lst[0][i]) & ('Final Status/ Number recovered' not in cln_lst[0][i]) == True:
                    cln_lst[0][i] = 'Number_Recovered'

    return cln_lst

def cln_trans(x):
    if 'None' in str(x):
        x = 'N'
    elif 'Yes' in str(x):
        x = 'Y'
    else:
        x = 'To be determined'
    return x

def cln_num_col(x):
    x = str(x).replace(' ','').replace('§','').replace('*','')
    if x == '':
        x = '0'
    return x

def cln_Country(x):
    x = x.lstrip().rstrip().split('^')[0]
    return x

# # define function to get df
def get_df(data,date):
    df = pd.DataFrame(data=data[1:],columns=data[0])
    df['Date'] = date
    return df



In [6]:
origin = list(map(get_data,url_use,data_date))

ValueError! Check source. http://www.who.int/csr/sars/country/table/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_10/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_11/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_12/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_14/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_15/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_16/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_17/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_18/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_19/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_21/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_22/en/
ValueError! Check source. http://www.who.int/csr/sars/country/2003_04_23/en/
Valu

In [7]:
data_origin = [origin[i][0] for i in range(len(origin)) if type(origin[i])==list]
data_df = list(map(clean_col,data_origin))

In [221]:
date_df = [origin[i][1] for i in range(len(origin)) if type(origin[i])==list]
lista = list(map(get_df, data_df, date_df))

In [357]:
dfa = pd.concat(lista)
dfb = dfa[dfa.columns[:5]]
dfb['Cumulative_Number_Of_Case(s)'] = list(map(cln_num_col,dfb['Cumulative_Number_Of_Case(s)']))
dfb['Number_Of_Deaths'] = list(map(cln_num_col,dfb['Number_Of_Deaths']))
dfb['Local_Chain(s)_of_Transmission'] = list(map(cln_trans,dfb['Local_Chain(s)_of_Transmission']))
dfb['Cumulative_Number_Of_Case(s)']=dfb['Cumulative_Number_Of_Case(s)'].astype(int)
dfb['Number_Of_Deaths']=pd.Series(list(map(lambda x: '0' if x =='nan' else x, dfb['Number_Of_Deaths']))).astype(int)
dfb['Country'] = list(map(cln_Country,dfb['Country']))
dfb['Country'] = list(map(lambda x: 'China' if 'China' in x else x, dfb['Country']))
dfb.reset_index(inplace=True)
dfb.drop(columns='index',inplace=True)
dfb.drop(dfb.index[[1585,1586]],inplace=True)
# dfb

In [356]:
dfc=dfb
totalindex = dfc[dfc.Country=='Total'].index
dfc.drop(totalindex , inplace=True)
# dfc # without total num
# totalindex = dfb[dfb.Country=='Total'].index
# totalindex

In [355]:
# dfc['Local_Chain(s)_of_Transmission'].value_counts() # column to be dropped

In [354]:
ts = dfc.groupby(['Date','Country']).agg('sum')
# ts

In [351]:
date = []
country = []
positive = []
death = []
for i in range(len(ts.index)):
    date.append(ts.index[i][0])
    country.append(ts.index[i][1])
    positive.append(ts.values[i][0])
    death.append(ts.values[i][1])
data = [date,country,positive,death]

In [353]:
new_df = pd.DataFrame(data=data,index=['Date','Country', 'Cumulative_Number_Of_Case(s)', 'Number_Of_Deaths']).T
# new_df.groupby('Date').agg('sum')

In [358]:
new_df.columns

Index(['Date', 'Country', 'Cumulative_Number_Of_Case(s)', 'Number_Of_Deaths'], dtype='object')

In [360]:
new_df.shape

(2071, 4)

In [361]:
# debug for urls raised error 
url_lst_structure2 = ['http://www.who.int/csr/sars/country/table/en/','http://www.who.int/csr/sars/country/2003_04_10/en','http://www.who.int/csr/sars/country/2003_04_11/en','http://www.who.int/csr/sars/country/2003_04_12/en','http://www.who.int/csr/sars/country/2003_04_14/en','http://www.who.int/csr/sars/country/2003_04_15/en','http://www.who.int/csr/sars/country/2003_04_16/en','http://www.who.int/csr/sars/country/2003_04_17/en','http://www.who.int/csr/sars/country/2003_04_18/en','http://www.who.int/csr/sars/country/2003_04_19/en','http://www.who.int/csr/sars/country/2003_04_21/en','http://www.who.int/csr/sars/country/2003_04_22/en','http://www.who.int/csr/sars/country/2003_04_23/en','http://www.who.int/csr/sars/country/table2003_09_23/en','http://www.who.int/csr/sars/country/table2004_04_21/en/']
url_use2 = [i for i in url_lst_structure2 if len(bs(requests.get(i).content,'html').find_all('tr'))!=0]
data_date2 = list(map(lambda x:re.findall('\d+_\d+_\d+',x)[0],url_use2[3:]))
date_lst2 = ['2003_03_17','2003_03_18','2003_03_19']

#df date col data
data_date2 = date_lst2+data_date2

In [771]:
# functions
def get_data2(url,date):
    soup_table2 = bs(requests.get(url).content,'html').find_all('tr')
    rows2 = []
    for row in soup_table2:
        rows2.append([item.text.replace(u'\xa0',u' ').strip('\n') for item in row.find_all('td')]) 
        df2 = pd.DataFrame(rows2)
        df2.rename(columns=pd.Series(df2.iloc[0].values),inplace=True)
        df2 = df2[1:]
        df2['Date'] = date
        result = [df2.values,df2.columns]
    return result
    
def cln_col2(x):
    cln_lst = list(x[1])
    for i in range(len(cln_lst)):
        if cln_lst[i] != None:
            if 'transmission' in cln_lst[i]:
                cln_lst[i] = 'Local_Chain(s)_of_Transmission'
            elif 'Country' in cln_lst[i]:
                cln_lst[i] = 'Country'
            elif ('Cumulative number' in cln_lst[i]) & ('Date for which cumulative number of cases is current' not in cln_lst[i]) == True:
                cln_lst[i] = 'Cumulative_Number_Of_Case(s)'
            elif ('Number of deaths' in cln_lst[i]) & ('Final Status/ Number of deaths' not in cln_lst[i]) == True:
                cln_lst[i] = 'Number_Of_Deaths'
            elif ('Number recovered' in cln_lst[i]) & ('Final Status/ Number recovered' not in cln_lst[i]) == True:
                cln_lst[i] = 'Number_Recovered'
        else:
            str(cln_lst[i])

    return cln_lst

def rmvNone(x):
    y = x
    for i in range(len(y)):
        if y[i] is None:
            y[i] = '0'
        elif y[i] == '':
            y[i] = '0'
    return y

In [774]:
origin2 = list(map(get_data2,url_use2,data_date2))

col_df2 = list(map(rmvNone, list(map(cln_col2, origin2))))
# [clean_col(origin2[i][1]) for i in range(len(origin2))]# if type(origin2[i])==list]


In [613]:

# col_df2 = list(map(clean_col,col_df2))
data_df2 = [origin2[i][0] for i in range(len(origin2))]# if type(origin2[i])==list]
# a_lst2 = [origin2[i][2] for i in range(len(origin2))]
# for i in range(len(data_df2)):
#     a_lst2.append(pd.DataFrame(data_df2[i]))
#     a_lst2[i]['Date'] = date_df2[i]
# dfa2 = pd.concat(a_lst2)

In [777]:
df2a = pd.concat([pd.DataFrame(data_df2[i]) for i in range(len(data_df2))])
df2a = df2a[df2a.columns[:5]]#.rename(columns=col_df2)
# col_df2

In [378]:
run = []
while i+1 <= len(data_origin2):
    for i in range(len(data_origin2)):
        if len(data_origin2[i])!=len(data_origin2[i+1]):
            print('in data_origin (index),  %s len not equals to %s' % (i, i+1))
        else: 
            run.append(i)

in data_origin (index),  0 len not equals to 1
in data_origin (index),  1 len not equals to 2
in data_origin (index),  3 len not equals to 4
in data_origin (index),  4 len not equals to 5
in data_origin (index),  6 len not equals to 7
in data_origin (index),  12 len not equals to 13


IndexError: list index out of range

In [785]:
df2a.rename(columns=)

,0,1,2,3,4
0,Germany,1,0,None*,2003_03_17
1,Canada,8,2,Yes,2003_03_17
2,Singapore,20,0,Yes,2003_03_17
3,Hong Kong Special Administrative Region of Ch...,95,1**,Yes,2003_03_17
4,Switzerland,2,0,To be determined,2003_03_17
5,Thailand,1,0,None*,2003_03_17
6,Viet Nam,40,1,Yes,2003_03_17
7,Total,167,4,,2003_03_17
0,Number of deaths,Number recovered,None,None,None
1,Brazil,2,1,0,0


In [784]:
df2a['Cumulative_Number_Of_Case(s)'] = list(map(cln_num_col,df2a['Cumulative_Number_Of_Case(s)']))
df2a['Number_Of_Deaths'] = list(map(cln_num_col,df2a['Number_Of_Deaths']))
df2a['Local_Chain(s)_of_Transmission'] = list(map(cln_trans,df2a['Local_Chain(s)_of_Transmission']))
df2a['Cumulative_Number_Of_Case(s)']=df2a['Cumulative_Number_Of_Case(s)'].astype(int)
df2a['Number_Of_Deaths']=pd.Series(list(map(lambda x: '0' if x =='nan' else x, df2a['Number_Of_Deaths']))).astype(int)
df2a['Country'] = list(map(cln_Country,df2a['Country']))
df2a['Country'] = list(map(lambda x: 'China' if 'China' in x else x, df2a['Country']))
# df2a.reset_index(inplace=True)
# df2a.drop(columns='index',inplace=True)
# df2a.drop(df2a.index[[1585,1586]],inplace=True)

KeyError: 'Cumulative_Number_Of_Case(s)'

In [515]:
url_lst_structure2

['http://www.who.int/csr/sars/country/table/en/',
 'http://www.who.int/csr/sars/country/2003_04_10/en',
 'http://www.who.int/csr/sars/country/2003_04_11/en',
 'http://www.who.int/csr/sars/country/2003_04_12/en',
 'http://www.who.int/csr/sars/country/2003_04_14/en',
 'http://www.who.int/csr/sars/country/2003_04_15/en',
 'http://www.who.int/csr/sars/country/2003_04_16/en',
 'http://www.who.int/csr/sars/country/2003_04_17/en',
 'http://www.who.int/csr/sars/country/2003_04_18/en',
 'http://www.who.int/csr/sars/country/2003_04_19/en',
 'http://www.who.int/csr/sars/country/2003_04_21/en',
 'http://www.who.int/csr/sars/country/2003_04_22/en',
 'http://www.who.int/csr/sars/country/2003_04_23/en',
 'http://www.who.int/csr/sars/country/table2003_09_23/en',
 'http://www.who.int/csr/sars/country/table2004_04_21/en/']